# Section 3

In [1]:
from PIL import Image
import numpy as np
from scipy.signal import convolve2d

def bw_threshold(img, t):
    return (img > t).astype(np.uint8) * 255

def rmse(f, b):
    f = np.double(f)
    b = np.double(b)
    
    sum = 0
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            sum += (f[i,j] - b[i,j])**2 
    return np.sqrt((1/(f.shape[0]*f.shape[1]))*sum)

def fidelity(f, b):
    fl = 255 * (f/255)**2.2
    bl = 255 * (b/255)**2.2
    
    h = np.zeros([7, 7])
    for i in range(7):
        for j in range(7):
            h[i, j] = np.exp(-((i-3)**2 + (j-3)**2)/(2*2))
    h = h / np.sum(h)
            
    f_tilde = 255 * (convolve2d(fl, h, mode='same', boundary='fill', fillvalue=0)/ 255)**(1/3)
    b_tilde = 255 * (convolve2d(bl, h, mode='same', boundary='fill', fillvalue=0)/ 255)**(1/3)

    return rmse(f_tilde, b_tilde)


input_img = np.array(Image.open('house.tif'))
binary_img = bw_threshold(input_img, 127)
binary_tif = Image.fromarray(binary_img)
binary_tif.save('img_out.tif')


print(f"RMSE: {rmse(input_img, binary_img)}")
print(f"Fidelity: {fidelity(input_img, binary_img)}")

RMSE: 87.3933165438293
Fidelity: 77.33714917243245


# Section 4

In [13]:
from PIL import Image
import numpy as np
from scipy.signal import convolve2d
from tabulate import tabulate

def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

def bw_threshold(img, t):
    """
    Compute a black and white version of the given img (2d np array) given threshold t.
    Returns: 2D np array with values of 0 and 255 corresponding to black and white pixels.
    """
    return (img > t).astype(np.uint8) * 255

def rmse(f, b):
    """
    Compute the root-mean square error between the two images.
    """
    return np.sqrt(1/(f.shape[0]*f.shape[1]) * np.sum((f-b)**2))

def fidelity(f, b):
    f_tilde = transform(f)
    b_tilde = transform(b)
    # The formula is the same, just pass in f_tilde and b_tilde to rmse
    return rmse(f_tilde, b_tilde)

def transform(img):
    # Ungamma correct
    gamma = 2.2
    fl = 255 * (img/255)**gamma
    # Construct the filter we wish to apply
    h = np.zeros([7, 7])
    sigma_squared = 2
    for i in range(7):
        for j in range(7):
            h[i, j] = np.e**(-((i-3)**2 + (j-3)**2)/(2*sigma_squared))
    # Make it so that the sum of h adds to 1 (handles the C coefficient)
    h = h / np.sum(h)
    filtered_img = convolve2d(fl, h, mode='same', boundary='fill', fillvalue=0)
    return 255 * (filtered_img / 255)**(1/3)


input_img = np.array(Image.open('house.tif'))
gamma = 2.2
input_linear = 255 * (input_img/255)**gamma
I2 = np.array([[1, 2], [3, 0]])
# I2N=np.block([[4*IN + 1,4*IN + 2],[4*IN + 3,4*IN]])
I4=np.block([[4*I2 + 1,4*I2 + 2],[4*I2 + 3,4*I2]])
I8=np.block([[4*I4 + 1,4*I4 + 2],[4*I4 + 3,4*I4]])
# print(I2)
# print(I4)
# print(I8)
print(bmatrix(I2))
print(bmatrix(I4))
print(bmatrix(I8))
T2 = 255 * (I2 + 0.5) / (2**2)
T4 = 255 * (I4 + 0.5) / (4**2)
T8 = 255 * (I8 + 0.5) / (8**2)
b2 = np.zeros(input_linear.shape)
b4 = np.zeros(input_linear.shape)
b8 = np.zeros(input_linear.shape)
bs = [b2, b4, b8]
Ts = [T2, T4, T8]
for index, b in enumerate(bs):
    N = Ts[index].shape[0]
    for i in range(input_linear.shape[0]):
        for j in range(input_linear.shape[1]):
            b[i, j] = 255 if input_linear[i, j] > Ts[index][i%N, j%N] else 0

# TODO: Determine if error and fidelity should be relative to linear image or input image
img_out2 = Image.fromarray(b2.astype(np.uint8))
img_out2.save("4-b2x2.tif")
print(f"Error 2x2: {rmse(input_img, b2)}")
print(f"Fidelity 2x2: {fidelity(input_img, b2)}")

img_out4 = Image.fromarray(b4.astype(np.uint8))
img_out4.save("4-b4x4.tif")
print(f"Error 4x4: {rmse(input_img, b4)}")
print(f"Fidelity 4x4: {fidelity(input_img, b4)}")

img_out8 = Image.fromarray(b8.astype(np.uint8))
img_out8.save("4-b8x8.tif")
print(f"Error 8x8: {rmse(input_img, b8)}")
print(f"Fidelity 8x8: {fidelity(input_img, b8)}")

\begin{bmatrix}
  1 & 2\\
  3 & 0\\
\end{bmatrix}
\begin{bmatrix}
  5 & 9 & 6 & 10\\
  13 & 1 & 14 & 2\\
  7 & 11 & 4 & 8\\
  15 & 3 & 12 & 0\\
\end{bmatrix}
\begin{bmatrix}
  21 & 37 & 25 & 41 & 22 & 38 & 26 & 42\\
  53 & 5 & 57 & 9 & 54 & 6 & 58 & 10\\
  29 & 45 & 17 & 33 & 30 & 46 & 18 & 34\\
  61 & 13 & 49 & 1 & 62 & 14 & 50 & 2\\
  23 & 39 & 27 & 43 & 20 & 36 & 24 & 40\\
  55 & 7 & 59 & 11 & 52 & 4 & 56 & 8\\
  31 & 47 & 19 & 35 & 28 & 44 & 16 & 32\\
  63 & 15 & 51 & 3 & 60 & 12 & 48 & 0\\
\end{bmatrix}
Error 2x2: 97.66897219213996
Fidelity 2x2: 50.05694796030144
Error 4x4: 101.00692201569473
Fidelity 4x4: 16.558342510690387
Error 8x8: 100.91452962396079
Fidelity 8x8: 14.691773433816369


# Section 5

In [8]:
from PIL import Image
import numpy as np
from scipy.signal import convolve2d


def rmse(f, b):
    f = np.double(f)
    b = np.double(b)
    
    sum = 0
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            sum += (f[i,j] - b[i,j])**2 
    return np.sqrt((1/(f.shape[0]*f.shape[1]))*sum)

def fidelity(f, b):
    fl = 255 * (f/255)**2.2
    bl = 255 * (b/255)**2.2
    
    h = np.zeros([7, 7])
    for i in range(7):
        for j in range(7):
            h[i, j] = np.exp(-((i-3)**2 + (j-3)**2)/(2*2))
    h = h / np.sum(h)
            
    f_tilde = 255 * (convolve2d(fl, h, mode='same', boundary='fill', fillvalue=0)/ 255)**(1/3)
    b_tilde = 255 * (convolve2d(bl, h, mode='same', boundary='fill', fillvalue=0)/ 255)**(1/3)

    return rmse(f_tilde, b_tilde)

def diffusion_error(corrected_image):
    dim1, dim2 = corrected_image.shape
    output_matrix = np.zeros((dim1, dim2))
    output_matrix = np.pad(output_matrix, ((1, 1), (1, 1)))
    corrected_image = np.pad(corrected_image, ((1, 1), (1, 1)))

    for row in range(1, dim1 + 1):
        for col in range(1, dim2 + 1):
            if corrected_image[row, col] > 127:
                output_matrix[row, col] = 255
            else:
                output_matrix[row, col] = 0
            error = corrected_image[row, col] - output_matrix[row, col]
            pos = [(row + 1, col - 1), (row + 1, col), (row + 1, col + 1), (row, col + 1)]
            weight = [3 / 16, 5 / 16, 1 / 16, 7 / 16]
            for k in range(len(pos)):
                corrected_image[pos[k]] += error * (weight[k])

    output_matrix = output_matrix[1:-1, 1:-1]

    return output_matrix


im = Image.open('house.tif')
input_img = np.array(im)
corrected_image = 255 * ((input_img / 255) ** 2.2)

output_img = diffusion_error(corrected_image)

img_out = Image.fromarray(output_img.astype(np.uint8))
img_out.save('output.tif')

print(f"Error: {rmse(input_img, output_img)}")
print(f"Fidelity: {fidelity(input_img, output_img)}")

Error: 98.84711671109255
Fidelity: 13.427253039026652
